<a href="https://colab.research.google.com/github/chicot1k/programming/blob/main/spade_local_voating_protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.4/387.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.4 MB/s eta 0:00:00
  Created wheel for aioxmpp: filename=aioxmpp-0.13.3-py3-none-any.whl size=47149

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from random import uniform
import time
import datetime
from asyncio import sleep as sl
from spade import run
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour
from spade.message import Message
from math import fabs, sqrt
import pandas as pd

# Данные существующих агентов
agents_info = [
    ("chicot1k@movim.eu", "pmpuagent"),
    ("chicot2k@movim.eu", "pmpuagent"),
    ("chicot3k@movim.eu", "pmpuagent"),
    ("chicot4k@movim.eu", "pmpuagent"),
    ("chicot5k@movim.eu", "pmpuagent"),
]

# Граф для 5 агентов (индексы соседей)
graph = [
    [1, 2],  # chicot1k соседи: chicot2k, chicot3k
    [0, 3],  # chicot2k соседи: chicot1k, chicot4k
    [0, 4],  # chicot3k соседи: chicot1k, chicot5k
    [1, 4],  # chicot4k соседи: chicot2k, chicot5k
    [2, 3],  # chicot5k соседи: chicot3k, chicot4k
]

class MyAgent(Agent):
    def __init__(self, neighbours=[], value: int = 0, start_time: datetime = datetime.datetime.now(), *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.neighs = neighbours
        self.state = value
        self.alpha = 0.2
        self.messages = []
        self.control = 0
        self.curr = 0
        self.start_time = start_time

    class Sender(PeriodicBehaviour):
        async def run(self):
            for n in self.agent.neighs:
                msg = Message(to=n)  # Сообщение для соседа
                if uniform(0, 1) > 0.05:
                    msg.body = str(self.agent.state + uniform(-1, 1))
                else:
                    msg.body = None
                await sl(uniform(0, 0.1))  # Случайная задержка
                await self.send(msg)
            if self.counter == limit:
                self.kill()
            self.counter += 1

        async def on_end(self):
            await self.agent.stop()

        async def on_start(self):
            self.counter = 0

    class Receiver(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                self.agent.curr += 1
                if msg.body is not None:
                    self.agent.messages.append(float(msg.body) - self.agent.state)
            if self.agent.curr == len(self.agent.neighs):
                for i in self.agent.messages:
                    self.agent.control += i
                self.agent.state += self.agent.alpha * self.agent.control
                self.agent.control = 0
                self.agent.messages = []
                self.agent.curr = 0

    async def setup(self):
        time.sleep(5)  # Небольшая задержка перед запуском
        self.add_behaviour(self.Receiver())
        self.add_behaviour(self.Sender(period=1, start_at=self.start_time))


limit = 1000

async def main():
    s = 0
    agents = []
    values = []
    start_at = datetime.datetime.now() + datetime.timedelta(seconds=10)  # Старт через 30 секунд
    for i, (jid, password) in enumerate(agents_info):
        value = uniform(0, 100)
        values.append(value)
        s += value
        agent = MyAgent(
            [agents_info[n][0] for n in graph[i]], value, start_at, jid, password, verify_security=False
        )
        await agent.start(auto_register=False)
        agents.append(agent)

    print(f'Average initial state = {s / len(agents):.2f}')

    # Ожидание завершения работы агентов
    await agents[-1].stop()
    for agent in agents:
        await agent.stop()

    data = {
        "agent": [str(i.name).split("@")[0] for i in agents],
        "initial state": [value for value in values],
        "final state": [i.state for i in agents],
        "error": [fabs(i.state - s / len(agents)) for i in agents],
    }
    print(pd.DataFrame(data))

    print(f'Default variance = {sqrt(sum([(i - s / len(agents)) ** 2 for i in values]) / len(agents)):.2f}')
    print(f"Final variance = {sqrt(sum([(i.state - s / len(agents)) ** 2 for i in agents]) / len(agents)):.2f}")


await main()
print('Done.')


51.05818229795247
</message>
42.89768204707187
</message>
43.98272700482781
</message>
47.86665661265137
</message>
82.12526948722018
</message>
46.030115675278736
</message>
46.70599112612522
</message>
82.10943174257056
</message>
94.03018579939899
</message>
69.0240178451445
</message>
92.69462544066123
</message>
65.97075607757316
</message>


Average initial state = 66.86
      agent  initial state  final state      error
0  chicot1k      51.039120    62.956129   3.901667
1  chicot2k      43.281561    67.387654   0.529858
2  chicot3k      82.565378    65.416850   1.440946
3  chicot4k      93.098135    78.812188  11.954393
4  chicot5k      64.304785    64.304785   2.553011
Default variance = 18.70
Final variance = 5.78
Done.
